In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# 카이제곱검정

In [2]:
N = 10  # 데이터 갯수
K = 4   # 범주 갯수
mu_0 = np.ones(K) / K   # np.ones : 모든 요소의 값이 1인 ndarray 만들기 -> 발생 확률(임의로 만든것)
# (K개=4개) 각 1, 4개가 4로 나뉘어 0.25 4개 됨
print(mu_0)

np.random.seed(42)
x = np.random.choice(K, N, p=mu_0)
# 랜덤 시드 - 재현성 - 난수가 발생해도 같은 난수가 발생하게 하는 것
# 컴퓨터는 뽑을때 규칙적이라 뽑힐 수를 흔들어 놓아서 난수 발생되게 한다
# 기본 시드 값을 현재 시간으로 해서 매번 달라지는 것

# x = np.random.choice(K, N, p=mu_0)
# K범주 안에서 (4개) N개(10개)를 뽑고, p=mu_0는 확률(25%)
# np.random.choice 구글링 - 넘파이 공식 문서 보기
# https://numpy.org/doc/2.1/reference/random/generated/numpy.random.choice.html

print(x)

c = np.bincount(x)
print(c)
# array([3, 1, 4, 2]) > 0이 3개, 1이 1개, 2가 4개, 3이 2개 라는 뜻

print(stats.chisquare(c))
print(stats.chisquare([7, 1, 1, 1]))
# pvalue=np.float64(0.5009344477491052)
# 각 동일 분표일때 불규칙 샘플이 뽑힐 확률
# 샘플이 같아도 50% 매우 빈번하다

[0.25 0.25 0.25 0.25]
[1 3 2 2 0 0 0 3 2 2]
[3 1 4 2]
Power_divergenceResult(statistic=np.float64(2.0), pvalue=np.float64(0.5724067044708798))
Power_divergenceResult(statistic=np.float64(10.8), pvalue=np.float64(0.012858001156297613))


In [6]:
N = 10  # 데이터 갯수
K = 4   # 범주 갯수
mu_0 = np.ones(K) / K 
print(mu_0)

np.random.seed(42)
x = np.random.choice(K, 10000000, p=mu_0)
# 샘플이 커질수록 0.25, 0.25, 0.25, 0.25에 근접하다
print(x)

c = np.bincount(x)
print(c)

print(stats.chisquare(c))
print(stats.chisquare([7, 1, 1, 1]))

[0.25 0.25 0.25 0.25]
[1 3 2 ... 3 0 1]
[2499927 2499831 2500938 2499304]
Power_divergenceResult(statistic=np.float64(0.5592600000000001), pvalue=np.float64(0.9056921541012419))
Power_divergenceResult(statistic=np.float64(10.8), pvalue=np.float64(0.012858001156297613))


In [8]:
ad_df = pd.read_csv('data-files/ad.csv')
ad_df

,광고,구입
0,B,하지 않았다
1,B,하지 않았다
2,A,했다
3,A,했다
4,B,하지 않았다
...,...,...
995,B,하지 않았다
996,B,하지 않았다
997,B,하지 않았다
998,B,하지 않았다


In [11]:
# ad_stats = ad_df.groupby('광고')["구입"].count().reset_index()
# 광고를 기준으로 구입이에 대한 카운트 구하기 > 전체 갯수 구하기
ad_stats = ad_df.groupby('광고')["구입"].value_counts().reset_index() # 값 별 갯수
ad_stats

,광고,구입,count
0,A,하지 않았다,351
1,A,했다,49
2,B,하지 않았다,549
3,B,했다,51


In [ ]:
# 49는 어느정도의 비중일까? > 1행 2열의 값임
# A 광고 구매율 구하기
print( ad_stats.iloc[1, 2] / ad_stats.iloc[:2, 2].sum() )
# 결과 0.1225 > 12.25%

# B 광고 구매율 구하기
print( ad_stats.iloc[3, 2] / ad_stats.iloc[2:, 2].sum() )
# 결과 0.085 > 8.5%

0.1225
0.085


In [26]:
# 교차표 작성 1 - 피벗테이블 사용
# index: 행 지정, columns: 열 지정, values: 값 지정
ad_cross1 = ad_stats.pivot_table(index="광고", columns="구입", values="count")
print( ad_cross1 )

# 교차표 작성 2 - 데이터프레임 함수 사용
ad_cross2 = pd.crosstab(ad_df['광고'], ad_df['구입'])
print( ad_cross2 )


구입  하지 않았다    했다
광고              
A    351.0  49.0
B    549.0  51.0
구입  하지 않았다  했다
광고            
A      351  49
B      549  51


In [ ]:
# chi-square test (카이 제곱 검정)
# stats.chi2_contingency(크로스테이블_이름)
stats.chi2_contingency(ad_cross1)

# 결과
# statistic=np.float64(3.344907407407407)
# pvalue=np.float64(0.06741327136707487)
# 이런 샘플(a=12.25%, b=8.5%)이 나올 확률로, 6%대(5%미만이 아님) 의미있는 차이는 없다고 판단
# dof=1
# expected_freq=array([[360.,  40.], [540.,  60.]

Chi2ContingencyResult(statistic=np.float64(3.344907407407407), pvalue=np.float64(0.06741327136707487), dof=1, expected_freq=array([[360.,  40.],
       [540.,  60.]]))

In [27]:
# chi-square test (카이 제곱 검정)
# stats.chi2_contingency(크로스테이블_이름)
stats.chi2_contingency(ad_cross2)


Chi2ContingencyResult(statistic=np.float64(3.344907407407407), pvalue=np.float64(0.06741327136707487), dof=1, expected_freq=array([[360.,  40.],
       [540.,  60.]]))